# The PSyclone Internal Representation (PSyIR) Example 1 - Basics

This example describes the structure of the PSyIR and how it may be traversed and searched.

The PSyIR is a canonical, language-agnostic, tree-based representation of code. To elaborate: it is 'canonical' in the sense that complex constructs are represented using their constituent parts. e.g. rather than represent `else if` directly in the PSyIR, such a control structure is represented as an `if` within an `else` clause. 'Language agnostic' means that, in principle, the PSyIR may be used to describe code written in any procedural programming language. Thus, language specific constructs (such as Fortran's WHERE) are mapped into generic PSyIR constructs. More details on this may be found in the PSyclone [Developer Guide](https://psyclone-dev.readthedocs.io/en/latest/psyir.html).

Each node in the tree 
is represented by a (sub-class of) the `Node` [class](https://psyclone-ref.readthedocs.io/en/latest/autogenerated/psyclone.psyir.nodes.html#psyclone.psyir.nodes.Node). As such, every node has the `parent` and `children` attributes as well as those particular to the specialised class. Any sequence of one or more nodes representing statements must have a `Schedule` as parent. Consequently, the bodies of e.g. a subroutine, loop or if statement all have a `Schedule` at their root. All of this is best illustrated by example.

We use the FortranReader frontend in order to generate some PSyIR with which to experiment. First, we need some simple Fortran code:

In [ ]:
code = '''program test
  implicit none
  integer, parameter :: jpi=10, jpj=10, jpk=10
  real, dimension(jpi,jpj,jpk) :: b
  integer :: ji,jj,jk
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        b(ji,jj,jk) = 0.0
      end do
    end do
  end do
end program test'''

Next, we use the FortranReader frontend to create the PSyclone Internal Representation of this code:

In [ ]:
from psyclone.psyir.frontend.fortran import FortranReader
freader = FortranReader()
psy = freader.psyir_from_source(code)
print(type(psy))

As can be seen, the root node of the PSyIR for this Fortran code is a `FileContainer`. As our simple code fragment contains a single routine (the program `test`), the `FileContainer` has a single `Routine` node as its child:

In [ ]:
psy.children

Every node in the PSyIR has a `view` method which returns a text-based representation of itself and its children:

In [ ]:
routine = psy.children[0]

routine.view()

Hopefully, the relationship between the structure of this PSyIR and our original Fortran code is fairly obvious. We have three (nested) loops, each of which has a start, a stop and an increment as its first three children. The body of each loop is contained within a `Schedule` node. Finally, inside the innermost loop, we have an `Assignment` node where an array reference is assigned the value 0.0.

As mentioned earlier, each node has the `parent` and `children` attributes. In this case the `Routine` has a single child (a `Loop`):

In [ ]:
print(routine.children)
loop = routine.children[0]

This Loop then has the Routine as parent and its loop bounds and body as children:

In [ ]:
print("Parent: ", type(loop.parent))
for child in loop.children:
    print("Child: ", type(child))

The most generic tool for searching for nodes within a tree is the [`walk`](https://psyclone.readthedocs.io/en/stable/psyir.html#psyclone.psyir.nodes.Node.walk) method. For instance, to find all of the `Loop` nodes within the top-level `Routine`:

In [ ]:
from psyclone.psyir import nodes
loops = routine.walk(nodes.Loop)
print(len(loops))

In [ ]:
for loop in loops:
    print(loop.variable)

Can you use the `walk` method to find all `Reference`s in the code and then print their `name`s?

As well as being able to `walk` down the tree of PSyIR nodes, it is also possible to search back *up* the tree using the `ancestor` method. If we were to find the `Assignment` statement:

In [ ]:
assign = routine.walk(nodes.Assignment)[0]

Then we could find the immediately enclosing `Loop` by doing:

In [ ]:
loop = assign.ancestor(nodes.Loop)
print(loop.variable)

Which tells us that this assignment is inside a loop over the `ji` variable.

Let's move on to the [next example](psyir_example2.ipynb)...